In [ ]:
import time
import sys
import os
import numpy as np
import npy2bdv

In [2]:
def generate_test_image(dim_yx, iz, nz):
    """Gaussian blob spanning the whole range of uint16 type"""
    x = np.linspace(-3, 3, dim_yx[1])
    y = np.linspace(-3, 3, dim_yx[0])
    sigma = 1.0 - abs(iz - nz/2) / nz
    x, y = np.meshgrid(x, y)
    return 65535 * np.exp(- ((x ** 2) + (y ** 2)) / (2 * sigma**2) )

examples_dir = "./example_files/"
if not os.path.exists(examples_dir):
    os.mkdir(examples_dir)

nz, ny, nx = 50, 1024, 2048
stack = np.empty((nz, ny, nx))
for z in range(nz):
    stack[z, :, :] = generate_test_image((ny, nx), z, nz)

## 1. Basic stack writing

In [7]:
fname = examples_dir + "ex1_t2_ch2_illum2_angle2.h5"
bdv_writer = npy2bdv.BdvWriter(fname, nchannels=2, nilluminations=2, nangles=2, subsamp=((1, 1, 1),))
for t in range(2):
    for i_ch in range(2):
        for i_illum in range(2):
            for i_angle in range(2):
                bdv_writer.append_view(stack, time=t, channel=i_ch, illumination=i_illum, angle=i_angle)

bdv_writer.write_xml_file(ntimes=2)
bdv_writer.close()
print(f"dataset in {fname}")

dataset in ./example_files/ex1_t2_ch2_illum2_angle2.h5


## 2. Writing speed test

Speed test for 20 time points and 2 channels. File size is 8 GB.

For some reason, stacks created in `float64` format are written **5x faster** than stacks in `uint16`. Storage in H5 is `uint16` in both cases, this is unexpected speed difference. Todo: look into h5py for details.

In [4]:
ntimes = 20
nchannels = 2
start_time_total = time.time()
i_stacks = 0
time_list = []
fname = examples_dir + "ex2_t20_chan2.h5"
bdv_writer = npy2bdv.BdvWriter(fname, nchannels=2, subsamp=((1, 1, 1),))
for ichannel in range(nchannels):
    for itime in range(ntimes):
        start_time = time.time()
        bdv_writer.append_view(stack, time=itime, channel=ichannel)
        time_interval = time.time() - start_time
        time_list.append(time_interval)
        i_stacks += 1.0

bdv_writer.write_xml_file(ntimes=ntimes)
bdv_writer.close()
time_per_stack = (time.time() - start_time_total) / i_stacks
print(f"H5 mean writing time per stack: {time_per_stack:1.3f} sec.")
print(f"H5 mean writing speed: {int(sys.getsizeof(stack) / time_per_stack / 1e6)} MB/s")
print(f"dataset in {fname}")

H5 mean writing time per stack: 0.361 sec.
H5 mean writing speed: 2321 MB/s
dataset in ./example_files/ex2_t20_chan2.h5


## 3. Writing with affine transformations defined in XML file

Affine transformations define translation, rotation, scaling, and shear.

In this example we write 1 time point and 1 channel with 10-px shear transformation along X axis. With non-isotropic voxel size calibration.
      
The affine transformation defined in XML file is automatically applied every time you open the dataset in BigDataViewer/BigStitcher.

In [5]:
shear_x_px = 10
affine_matrix = np.array(((1.0, 0.0, -shear_x_px, 0.0),
                          (0.0, 1.0, 0.0, 0.0),
                          (0.0, 0.0, 1.0, 0.0)))

fname = examples_dir + "ex3_t1_ch1_shear.h5"
bdv_writer = npy2bdv.BdvWriter(fname, nchannels=1, subsamp=((1, 1, 1),))
bdv_writer.append_view(stack, time=0, channel=0,
                       m_affine=affine_matrix,
                       name_affine="shearing transformation",
                       calibration=(1, 1, 1))
bdv_writer.write_xml_file(ntimes=1)
bdv_writer.close()
print(f"sheared dataset in {fname}")

sheared dataset in ./example_files/ex3_t1_ch1_shear.h5


## 3.1 Writing multiple tiles on a grid

Here, affine transformation matrix defines translation of individual tiles in the global coordinate system.
Tile position is the position of its lower left corner. The last (4-th) column of the affine matrix defines the translation terms (x,y,z).

Optionally, make some tiles missing.

In [5]:
DEBUG_MODE = False
MISSING_TILES = False

n_tiles_x = 4
n_tiles_y = 2

tile_w, tile_h = stack.shape[2] // n_tiles_x, stack.shape[1] // n_tiles_y

tile_position_yx = ((0,0), (0, tile_w), (tile_h, 0), (tile_h, tile_w))

unit_matrix = np.array(((1.0, 0.0, 0.0, 0.0), # change the 4. value for x_translation (px)
                        (0.0, 1.0, 0.0, 0.0), # change the 4. value for y_translation (px)
                        (0.0, 0.0, 1.0, 0.0)))# change the 4. value for z_translation (px)

fname = examples_dir + "ex3p1_tiles.h5"
bdv_writer = npy2bdv.BdvWriter(fname, ntiles=n_tiles_x*n_tiles_y)
for j in range(n_tiles_y):
    for i in range(n_tiles_x):
        tile_stack = stack[:, j*tile_h:(j+1)*tile_h, i*tile_w:(i+1)*tile_w]
        tile_index = i + j*n_tiles_x
        affine_matrix = unit_matrix
        affine_matrix[0,3] = i * tile_w # x-translation
        affine_matrix[1,3] = j * tile_h # y-translation
        if DEBUG_MODE:
            print(f"Tile index: {tile_index}")
            print(f"Transform matrix: \n {affine_matrix}")
        if MISSING_TILES and tile_index in (1,2): # make two tiles missing 
            pass
        else:
            bdv_writer.append_view(tile_stack, time=0, 
                                   tile=tile_index, 
                                   m_affine=affine_matrix, 
                                   name_affine=f"tile {tile_index} translation")
bdv_writer.write_xml_file(ntimes=1)
bdv_writer.close()
print(f"\n File with multiple tiles: {fname}")


 File with multiple tiles: ./example_files/ex3p1_tiles.h5


## 4. Writing with experiment metadata
Dataset with 1 time point and 1 channel with voxel size, exposure, camera and microscope properties stored in XML.

In [7]:
fname = examples_dir + "ex4_t1_ch1_cam_props.h5"
bdv_writer = npy2bdv.BdvWriter(fname, nchannels=1, subsamp=((1, 1, 1),))
bdv_writer.append_view(stack, time=0, channel=0,
                       voxel_size_xyz=(1, 1, 5), voxel_units='um',
                       exposure_time=10, exposure_units='ms')
bdv_writer.write_xml_file(ntimes=1, camera_name="Hamamatsu OrcaFlash100",
                          microscope_name='Superscope',
                          user_name='nvladimus')
bdv_writer.close()
print(f"dataset is in {fname}")

dataset is in ./example_files/ex4_t1_ch1_cam_props.h5


## 5. Subsampling and compression
Dataset contains 1 time point and 1 channel with 3-level subsampling and compression.

In [8]:
%%time
fname = examples_dir + "ex5_t1_ch1_level3_gzip.h5"
bdv_writer = npy2bdv.BdvWriter(fname, nchannels=1,
                               subsamp=((1, 1, 1), (2, 4, 4), (4, 16, 16)),
                               blockdim=((64, 64, 64),),
                               compression='gzip')
bdv_writer.append_view(stack, time=0, channel=0)
bdv_writer.write_xml_file(ntimes=1)
bdv_writer.close()
print(f"dataset is in {fname}")

INFO: blockdim levels (1) < subsamp levels (3): First-level block size (64, 64, 64) will be used for all levels
dataset is in ./example_files/ex5_t1_ch1_level3_gzip.h5
Wall time: 7.9 s


## 6. Virtual stacks that are too big to fit RAM
Dataset has 1 time point, 2 channels, and large virtual stack, 21 GB, written plane by plane.

No subsampling or compression here (see next example for these options)

In [9]:
%%time
nz, ny, nx = 250, 3648, 5472
test_image = generate_test_image((ny, nx), nz/2, nz)
fname = examples_dir + "ex6_t1_ch1_huge_virtual.h5"
bdv_writer = npy2bdv.BdvWriter(fname, nchannels=2, blockdim=((1, 256, 256),))

for i_ch in range(2):
    bdv_writer.append_view(stack=None, virtual_stack_dim=(nz,ny,nx), time=0, channel=i_ch)
    for z in range(nz):
        bdv_writer.append_plane(plane=test_image, plane_index=z, time=0, channel=i_ch)

bdv_writer.write_xml_file(ntimes=1)
bdv_writer.close()
print(f"virtual stack in {fname}")

virtual stack in ./example_files/ex6_t1_ch1_huge_virtual.h5
Wall time: 43.3 s


### 6.a Virtual stacks with subsampling

Subsampling allows faster viewing in BDV, because only the necessary level of details is loaded into RAM, instead of full resolution image.

IN virtual stack writing, the subsampling level Z must be 1, because the data is written plane by plane, so there is no way to subsample in Z on the fly. For example, `subsamp=((1, 1, 1), (1, 4, 4), (1, 8, 8))`.

The newly generated file is slightly bigger (by ca. 8%), but the processing time increases 5x.

Compression is supported in the same way as for normal stacks, see Example 5.

In [10]:
%%time
fname = examples_dir + "ex6a_t1_ch1_huge_virtual_subsamp.h5"
bdv_writer = npy2bdv.BdvWriter(fname, nchannels=2,
                               blockdim=((1, 128, 128),),
                               subsamp=((1, 1, 1), (1, 4, 4), (1, 8, 8)))

for i_ch in range(2):
    bdv_writer.append_view(stack=None, virtual_stack_dim=(nz,ny,nx), time=0, channel=i_ch)
    for z in range(nz):
        bdv_writer.append_plane(plane=test_image, plane_index=z, time=0, channel=i_ch)

bdv_writer.write_xml_file(ntimes=1)
bdv_writer.close()
print(f"virtual stack with subsampling is in {fname}")

INFO: blockdim levels (1) < subsamp levels (3): First-level block size (1, 128, 128) will be used for all levels
virtual stack with subsampling is in ./example_files/ex6a_t1_ch1_huge_virtual_subsamp.h5
Wall time: 4min 32s


## 7. Missing views, normal stack
Missing views are detected automatically and marked as such in XML file.

In [11]:
fname = examples_dir + "ex7_missing_views.h5"
bdv_writer = npy2bdv.BdvWriter(fname, nchannels=2, subsamp=((1, 1, 1),))
bdv_writer.append_view(stack, time=0, channel=0)
bdv_writer.append_view(stack, time=1, channel=1)
bdv_writer.write_xml_file(ntimes=2)
bdv_writer.close()
print(f"dataset with missing views in {fname}")

dataset with missing views in ./example_files/ex7_missing_views.h5


## 8. Missing views, virtual stack

In [12]:
fname = examples_dir + "ex8_virtual_stack_missing_views.h5"
bdv_writer = npy2bdv.BdvWriter(fname, nchannels=2, subsamp=((1, 1, 1),))
bdv_writer.append_view(stack=None, virtual_stack_dim=(nz, ny, nx), time=0, channel=0)
bdv_writer.append_view(stack=None, virtual_stack_dim=(nz, ny, nx), time=1, channel=1)

for z in range(nz):
    bdv_writer.append_plane(plane=test_image, plane_index=z, time=0, channel=0)
    bdv_writer.append_plane(plane=test_image, plane_index=z, time=1, channel=1)

bdv_writer.write_xml_file(ntimes=2)
bdv_writer.close()
print(f"dataset with missing views in {fname}")

dataset with missing views in ./example_files/ex8_virtual_stack_missing_views.h5


## Clean up the data files generated

In [8]:
import shutil
if os.path.exists(examples_dir):
    shutil.rmtree(examples_dir)